In [ ]:
import pandas as pd

# 데이터 불러오기
nh_data = pd.read_csv('NH_data.csv')
us_stock_data = pd.read_csv('US_stock_data.csv')
etf_data = pd.read_csv('ETF_data.csv')

# 데이터 병합 (예시: 공통 키인 티커종목코드로 병합)
merged_data = pd.merge(nh_data, us_stock_data, on='TCK_IEM_CD', how='inner')
merged_data = pd.merge(merged_data, etf_data, on='TCK_IEM_CD', how='inner')

# 결측치 확인
print(merged_data.isnull().sum())

# 결측치 대체 (예시: 평균값으로 대체)
merged_data.fillna(merged_data.mean(), inplace=True)

# 데이터 타입 변환 (예: 날짜 형식)
merged_data['BSE_DT'] = pd.to_datetime(merged_data['BSE_DT'], format='%Y%m%d')




In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# 기초 통계량
print(merged_data.describe())

# 평균 보유 기간 분포
sns.histplot(merged_data['TCO_AVG_HLD_TE_DD_CNT'], bins=30)
plt.title('평균 보유 기간 분포')
plt.xlabel('보유 기간 (일수)')
plt.ylabel('빈도수')
plt.show()



#### 2. **투자자 행동 분석**

투자자의 행동을 분석하여 장기 투자자와 단기 투자자의 특징을 파악합니다.

##### **2.1. 평균 보유 기간 분석 (TCO_AVG_HLD_TE_DD_CNT)**
- **목표**: 투자자들이 자산을 평균적으로 얼마나 오랫동안 보유하고 있는지 분석합니다.
- **방법**:
  - 보유 기간의 분포를 시각화합니다.
  - 보유 기간에 따라 투자자를 그룹화하고 각 그룹의 특성을 분석합니다.


In [ ]:

# 보유 기간 분포 시각화
sns.histplot(merged_data['TCO_AVG_HLD_TE_DD_CNT'], bins=30, kde=True)
plt.title('평균 보유 기간 분포')
plt.xlabel('보유 기간 (일수)')
plt.ylabel('빈도수')
plt.show()

# 보유 기간 기준 그룹화 (예: 단기 < 180일, 장기 >= 180일)
merged_data['Investment_Horizon'] = merged_data['TCO_AVG_HLD_TE_DD_CNT'].apply(lambda x: '단기' if x < 180 else '장기')

# 그룹별 투자자 수
horizon_counts = merged_data['Investment_Horizon'].value_counts()
sns.barplot(x=horizon_counts.index, y=horizon_counts.values)
plt.title('투자자별 투자 기간 분포')
plt.xlabel('투자 기간')
plt.ylabel('투자자 수')
plt.show()



##### **2.2. 고객 세그먼트 분석**
- **목표**: 투자자의 실력, 연령대, 자산 규모에 따른 투자 행동의 차이를 분석합니다.
- **방법**:
  - **CUS_CGR_LLF_CD**와 **CUS_CGR_MLF_CD**를 사용하여 투자자를 세그먼트로 분류합니다.
  - 각 세그먼트별로 주요 지표(평균 보유 기간, 수익률 등)를 분석합니다.


In [ ]:

# 고객 세그먼트별 평균 보유 기간
segment_analysis = merged_data.groupby(['CUS_CGR_LLF_CD', 'CUS_CGR_MLF_CD'])['TCO_AVG_HLD_TE_DD_CNT'].mean().reset_index()

# 시각화
sns.barplot(x='CUS_CGR_LLF_CD', y='TCO_AVG_HLD_TE_DD_CNT', hue='CUS_CGR_MLF_CD', data=segment_analysis)
plt.title('고객 세그먼트별 평균 보유 기간')
plt.xlabel('고객 구성 대분류 코드')
plt.ylabel('평균 보유 기간 (일수)')
plt.legend(title='고객 구성 중분류 코드')
plt.show()



##### **2.3. 투자 성과 분석**
- **목표**: 투자자들의 손실 비율과 수익 비율을 분석하여 전반적인 투자 성과를 파악합니다.
- **방법**:
  - **LSS_IVO_RT**와 **PFT_IVO_RT**를 사용하여 투자 성과를 분석합니다.
  - 투자자 세그먼트별로 손실과 수익 비율의 차이를 비교합니다.


In [ ]:

# 투자 성과 비율 시각화
sns.scatterplot(x='LSS_IVO_RT', y='PFT_IVO_RT', hue='Investment_Horizon', data=merged_data)
plt.title('손실 투자자 비율 vs 수익 투자자 비율')
plt.xlabel('손실 투자자 비율 (%)')
plt.ylabel('수익 투자자 비율 (%)')
plt.show()



#### 3. **유동성 및 변동성 분석**

주식의 유동성과 투자 성과를 분석하여 단기 및 장기 투자에 적합한 종목을 식별합니다.

##### **3.1. 종목 조회 건수와 관심 종목 등록 건수 분석**
- **목표**: 종목 조회 건수(VW_TGT_CNT)와 관심 종목 등록 건수(RGS_TGT_CNT)를 분석하여 유동성과 투자 관심도를 평가합니다.
- **방법**:
  - 두 지표 간의 상관관계를 분석합니다.
  - 높은 조회 건수와 등록 건수를 가진 종목을 식별합니다.


In [ ]:

# 상관관계 분석
correlation = merged_data['VW_TGT_CNT'].corr(merged_data['RGS_TGT_CNT'])
print(f'종목 조회 건수와 관심 종목 등록 건수의 상관계수: {correlation}')

# 상관관계 시각화
sns.scatterplot(x='VW_TGT_CNT', y='RGS_TGT_CNT', data=merged_data)
plt.title('종목 조회 건수 vs 관심 종목 등록 건수')
plt.xlabel('종목 조회 건수')
plt.ylabel('관심 종목 등록 건수')
plt.show()



##### **3.2. 주식의 수익률 분석**
- **목표**: 주식의 단기(1개월), 중기(3개월), 장기(1년) 수익률을 분석하여 투자 유형에 따른 성과를 평가합니다.
- **방법**:
  - **MM1_TOT_PFT_RT**, **MM3_TOT_PFT_RT**, **YR1_TOT_PFT_RT** 데이터를 사용하여 수익률을 분석합니다.
  - 투자자의 보유 기간과 수익률 간의 관계를 분석합니다.


In [ ]:

# 수익률 분포 시각화
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.histplot(merged_data['MM1_TOT_PFT_RT'], bins=30, kde=True)
plt.title('1개월 총수익률 분포')
plt.xlabel('수익률 (%)')

plt.subplot(1,3,2)
sns.histplot(merged_data['MM3_TOT_PFT_RT'], bins=30, kde=True)
plt.title('3개월 총수익률 분포')
plt.xlabel('수익률 (%)')

plt.subplot(1,3,3)
sns.histplot(merged_data['YR1_TOT_PFT_RT'], bins=30, kde=True)
plt.title('1년 총수익률 분포')
plt.xlabel('수익률 (%)')

plt.tight_layout()
plt.show()



#### 4. **시장 심리 분석**

시장 전반의 심리를 파악하여 투자자들의 공포와 낙관주의를 이해합니다.

##### **4.1. 손실 투자자 비율과 수익 투자자 비율 분석**
- **목표**: 전체 투자자 중 손실을 보고 있는 비율과 수익을 내고 있는 비율을 분석하여 시장의 공포와 낙관주의 수준을 평가합니다.
- **방법**:
  - 두 비율의 평균과 분포를 분석합니다.
  - 시간에 따른 변화 추이를 분석합니다.


In [ ]:

# 손실 및 수익 투자자 비율의 기초 통계

print(merged_data[['LSS_IVO_RT', 'PFT_IVO_RT']].describe())

# 시간에 따른 손실 및 수익 비율 변화 시각화
plt.figure(figsize=(12,6))
sns.lineplot(x='BSE_DT', y='LSS_IVO_RT', label='손실 투자자 비율')
sns.lineplot(x='BSE_DT', y='PFT_IVO_RT', label='수익 투자자 비율')
plt.title('시간에 따른 손실 및 수익 투자자 비율 변화')
plt.xlabel('날짜')
plt.ylabel('비율 (%)')
plt.legend()
plt.show()



##### **4.2. 신규 매수 계좌와 전량 매도 계좌 분석**
- **목표**: 신규 매수 계좌(IFW_ACT_CNT)와 전량 매도 계좌(OFW_ACT_CNT)의 변화를 분석하여 투자자들의 시장 진입 및 이탈 동향을 파악합니다.
- **방법**:
  - 두 지표의 시간에 따른 변화를 시각화합니다.
  - 상관관계를 분석하여 매수와 매도의 트렌드를 파악합니다.


In [ ]:

# 신규 매수 계좌와 전량 매도 계좌의 시간에 따른 변화 시각화
plt.figure(figsize=(12,6))
sns.lineplot(x='BSE_DT', y='IFW_ACT_CNT', label='신규 매수 계좌 수')
sns.lineplot(x='BSE_DT', y='OFW_ACT_CNT', label='전량 매도 계좌 수')
plt.title('시간에 따른 신규 매수 및 전량 매도 계좌 수 변화')
plt.xlabel('날짜')
plt.ylabel('계좌 수')
plt.legend()
plt.show()

# 상관관계 분석
correlation_accounts = merged_data['IFW_ACT_CNT'].corr(merged_data['OFW_ACT_CNT'])
print(f'신규 매수 계좌 수와 전량 매도 계좌 수의 상관계수: {correlation_accounts}')


#### 5. **전문가 투자자 추적**

고급 투자자들의 행동을 분석하여 시장에 미치는 영향을 파악합니다.

##### **5.1. 고급 투자자 활동 분석**
- **목표**: 고급 투자자(CUS_CGR_LLF_CD = "1")의 투자 활동을 추적하고, 이들의 투자 패턴이 시장 성과에 미치는 영향을 분석합니다.
- **방법**:
  - 고급 투자자들의 보유 자산, 거래 빈도, 수익률 등을 분석합니다.
  - 고급 투자자들이 주로 투자하는 종목과 그 성과를 평가합니다.


In [ ]:

# 고급 투자자 필터링
expert_investors = merged_data[merged_data['CUS_CGR_LLF_CD'] == 1]

# 고급 투자자의 평균 보유 기간
expert_avg_holding = expert_investors['TCO_AVG_HLD_TE_DD_CNT'].mean()
print(f'고급 투자자의 평균 보유 기간: {expert_avg_holding}일')

# 고급 투자자들이 투자하는 종목의 수익률 분석
sns.boxplot(x='Investment_Horizon', y='YR1_TOT_PFT_RT', data=expert_investors)
plt.title('고급 투자자별 1년 총수익률')
plt.xlabel('투자 기간')
plt.ylabel('1년 총수익률 (%)')
plt.show()



##### **5.2. 고급 투자자와 일반 투자자 비교**
- **목표**: 고급 투자자와 일반 투자자의 투자 행동 및 성과를 비교하여 차이점을 파악합니다.
- **방법**:
  - 두 그룹의 주요 지표(보유 기간, 수익률, 매수/매도 활동 등)를 비교 분석합니다.


In [ ]:

##### **5.2. 고급 투자자와 일반 투자자 비교**
- **목표**: 고급 투자자와 일반 투자자의 투자 행동 및 성과를 비교하여 차이점을 파악합니다.
- **방법**:
  - 두 그룹의 주요 지표(보유 기간, 수익률, 매수/매도 활동 등)를 비교 분석합니다.


In [ ]:

# 그룹별 주요 지표 비교
comparison_metrics = merged_data.groupby('CUS_CGR_LLF_CD')[['TCO_AVG_HLD_TE_DD_CNT', 'PFT_IVO_RT', 'LSS_IVO_RT']].mean().reset_index()

# 시각화
sns.barplot(x='CUS_CGR_LLF_CD', y='TCO_AVG_HLD_TE_DD_CNT', data=comparison_metrics)
plt.title('투자자 그룹별 평균 보유 기간')
plt.xlabel('고객 구성 대분류 코드')
plt.ylabel('평균 보유 기간 (일수)')
plt.show()

sns.barplot(x='CUS_CGR_LLF_CD', y='PFT_IVO_RT', data=comparison_metrics)
plt.title('투자자 그룹별 수익 투자자 비율')
plt.xlabel('고객 구성 대분류 코드')
plt.ylabel('수익 투자자 비율 (%)')
plt.show()

sns.barplot(x='CUS_CGR_LLF_CD', y='LSS_IVO_RT', data=comparison_metrics)
plt.title('투자자 그룹별 손실 투자자 비율')
plt.xlabel('고객 구성 대분류 코드')
plt.ylabel('손실 투자자 비율 (%)')
plt.show()


##### **6.1. Value Chain 상관관계 분석**
- **목표**: 동일 산업 내 경쟁자 간의 상관관계를 분석하여 시장 내 경쟁 구도를 이해합니다.
- **방법**:
  - 각 기업의 주가 변동, 거래량 등을 기반으로 상관관계를 계산합니다.
  - **상관계수 행렬**을 작성하고 시각화하여 경쟁자 간의 관계를 파악합니다.


In [ ]:

# 기업별 주가 변동률 계산
merged_data['Price_Change'] = merged_data['IEM_END_PR'] - merged_data['IEM_ONG_PR']

# 기업별 평균 주가 변동률 피벗
pivot_data = merged_data.pivot_table(index='BSE_DT', columns='TCK_IEM_CD', values='Price_Change')

# 상관계수 행렬 계산
correlation_matrix = pivot_data.corr()

# 상관계수 히트맵 시각화
plt.figure(figsize=(12,10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('기업 간 주가 변동률 상관관계 히트맵')
plt.show()


##### **6.2. 시장 예측 모델링**
- **목표**: 수집된 데이터를 바탕으로 시장의 향후 움직임을 예측하는 모델을 개발합니다.
- **방법**:
  - 머신러닝 알고리즘(예: 랜덤 포레스트, XGBoost)을 사용하여 예측 모델을 구축합니다.
  - 모델의 성능을 평가하고 최적화합니다.


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 피처 선택 및 타겟 설정
features = ['LSS_IVO_RT', 'PFT_IVO_RT', 'IFW_ACT_CNT', 'OFW_ACT_CNT', 'VW_TGT_CNT', 'RGS_TGT_CNT']
target = 'IEM_END_PR'

# 데이터 분할
X = merged_data[features]
y = merged_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'평균 제곱 오차(MSE): {mse}')

# 중요 변수 시각화
importances = model.feature_importances_
sns.barplot(x=importances, y=features)
plt.title('특징 중요도')
plt.xlabel('중요도')
plt.ylabel('특징')
plt.show()
```



#### 7. **결과 해석 및 인사이트 도출**

분석을 통해 얻은 결과를 해석하고, 투자자들에게 유용한 인사이트를 제공합니다.

- **상관관계 분석**: 특정 산업 내 경쟁자 간의 강한 상관관계는 경쟁 또는 협력 관계를 나타낼 수 있습니다. 이를 통해 시장 전략을 수립할 수 있습니다.
- **시장 심리 인사이트**: 손실 투자자 비율과 수익 투자자 비율을 통해 시장의 공포와 낙관주의 수준을 파악하고, 이를 기반으로 투자 전략을 조정할 수 있습니다.
- **유동성 트렌드**: 높은 조회 건수와 등록 건수를 가진 종목은 유동성이 높아 단기 및 장기 투자에 모두 적합할 수 있습니다.
- **전문가 투자자 추적**: 고급 투자자들의 투자 패턴을 분석하여 일반 투자자들이 참고할 수 있는 전략을 도출할 수 있습니다.
